In [ ]:
from google.colab import files
import os
import shutil

# Cria uma pasta (se não existir)
os.makedirs("imagens", exist_ok=True)

# Envia o arquivo
uploaded = files.upload()

# Move o(s) arquivo(s) para a pasta "imagens"
for nome_arquivo in uploaded.keys():
    shutil.move(nome_arquivo, os.path.join("imagens", nome_arquivo))

In [ ]:
# Extrai as imagens do arquivo zip
import zipfile

with zipfile.ZipFile("/content/imagens/archive.zip", 'r') as zip_ref:
    zip_ref.extractall("imagens")


In [ ]:
#Lista as pastas do arquivo zip
print(os.listdir('imagens'))

['train', 'test', 'archive.zip', 'test.csv', 'train.csv']
['train', 'test', 'archive.zip', 'test.csv', 'train.csv']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Normaliza as imagens, dividindo os valores dos pixels por 255 (transforma de [0,255] para [0,1]).
#Divide os dados em 2 partes: 80% para treino e 20% para validação (com validation_split=0.2).
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


#Lê as imagens da pasta /content/imagens/train.

#Redimensiona todas para 128x128 pixels.

#Agrupa as imagens em lotes de 32 (batch_size=32).

#class_mode='categorical' indica que o modelo terá mais de 2 classes (usa one-hot encoding nos rótulos).

#Usa os 80% das imagens como treino (subset='training').
train_generator = datagen.flow_from_directory(
    '/content/imagens/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)





#Lê imagens da pasta /content/imagens/test.

#Aplica o mesmo pré-processamento e separação.

#Aqui, vai usar os 20% das imagens como validação (subset='validation').
validation_generator = datagen.flow_from_directory(
    '/content/imagens/test',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 1013 images belonging to 4 classes.
Found 79 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
#Usa o modelo Sequential do Keras
model = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),#Cria 32 filtros (ou kernels) de tamanho 3x3.
  MaxPooling2D((2, 2)),#Reduz a dimensionalidade (tamanho da imagem) pegando o valor máximo de cada bloco 2x2.
  Conv2D(64, (3, 3), activation='relu'),#Adiciona 64 filtros 3x3.
  MaxPooling2D((2, 2)),#Reduz a dimensionalidade
  Flatten(), # Transfroma a saída em 1D
  Dense(128, activation='relu'), # Camada com 128 neurônios totalmente conectados.
  Dropout(0.5), # Desativa aleatoriamente 50% dos neurônios desta camada.
  Dense(4, activation='softmax') # Camada final com 4 neurônios, um para cada classe.
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,392,836 (28.20 MB)

 Trainable params: 7,392,836 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(train_generator, validation_data=validation_generator, epochs=10)
model.save('modelo.h5')

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - accuracy: 0.2853 - loss: 1.7016

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


32/32 ━━━━━━━━━━━━━━━━━━━━ 18s 407ms/step - accuracy: 0.2866 - loss: 1.6945 - val_accuracy: 0.6709 - val_loss: 1.2006
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 250ms/step - accuracy: 0.4678 - loss: 1.2149 - val_accuracy: 0.6203 - val_loss: 0.8509
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 274ms/step - accuracy: 0.5569 - loss: 1.0274 - val_accuracy: 0.7215 - val_loss: 0.8525
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 271ms/step - accuracy: 0.6298 - loss: 0.9052 - val_accuracy: 0.8608 - val_loss: 0.6238
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 248ms/step - accuracy: 0.7022 - loss: 0.6926 - val_accuracy: 0.7468 - val_loss: 0.6209
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 278ms/step - accuracy: 0.7934 - loss: 0.5657 - val_accuracy: 0.8228 - val_loss: 0.5074
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 263ms/step - accuracy: 0.8266 - loss: 0.4631 - val_accuracy: 0.8228 - val_loss: 0.4963
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 247ms/step - accuracy: 0.8606 - loss: 0.3816 - val_accuracy: 0.7595 - val

In [ ]:
uploaded_analise = files.upload()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
img_path = '/content/imagem1.jpeg'

img = image.load_img(img_path, target_size=(128,128))
img_array = image.img_to_array(img)

img_array = img_array/255.0

img_array = np.expand_dims(img_array, axis=0)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('modelo.h5')

In [ ]:
pred = model.predict(img_array)
print(pred)  # Vai mostrar as probabilidades para cada classe

# Pegar o índice da classe com maior probabilidade
pred_class = np.argmax(pred, axis=1)[0]

# Mapear o índice para o nome da classe
class_names = list(train_generator.class_indices.keys())  # pega classes do gerador

print("Doença detectada:", class_names[pred_class])


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step
[[1.7679378e-28 2.4472864e-18 1.0000000e+00 1.2169887e-17]]
Doença detectada: phoma
